In [1]:
import os
import sys
sys.path.append('../../')
from util import summarize_trial, log_trial, save_agents
import joblib
from react_cls import CoTAgent
from mocks import DocStoreExplorerMock, LLMMock
import numpy as np

#### Load the HotpotQA Sample

In [4]:
hotpot = joblib.load('data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

#### Initialize a CoTAgent for each question

In [5]:
from prompts import cot_simple_reflect_agent_prompt, cot_simple_reflect_prompt
from fewshots import COTQA_SIMPLE6, COT_SIMPLE_REFLECTION
agents = [CoTAgent(row['question'], '', row['answer'],
                   agent_prompt=cot_simple_reflect_agent_prompt,
                     reflect_prompt=cot_simple_reflect_prompt,
                     cot_examples = COTQA_SIMPLE6,
                        reflect_examples = COT_SIMPLE_REFLECTION) for _, row in hotpot.iterrows()]

#### Run Trials

In [7]:
trial = 0
log = ''
for i in range(6):
    for agent in [a for a in agents if not a.is_correct()]:
        agent.run(reflect_strategy='last_attempt + reflexion')
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_trial(agents, trial)
    correct, incorrect = summarize_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

Thought: Let's think step by step. VIVA Media AG changed its name to Viacom International Media Networks in 2004. The acronym for Viacom International Media Networks is VIMN, so the answer is VIMN.
Action: Finish[VIMN]
Answer: Gesellschaft mit beschränkter Haftung
Thought: Let's think step by step. Jonny Craig has been a member of six bands, while Pete Doherty has been a member of seven bands. So Pete Doherty has been a member of more bands.
Action: Finish[Pete Doherty]
Answer: Jonny" Craig
Thought: Let's think step by step. The Missouri Compromise was an agreement between the United States and the state of Missouri in 1820. The first governor after the Missouri Compromise was Alexander McNair, who was from Missouri. So the answer is Missouri.
Action: Finish[Missouri]
Answer: Bath, Maine
Thought: Let's think step by step. The creator of "Wallace and Gromit" is Nick Park. Nick Park created the animation comedy "Creature Comforts" which matched animated zoo animals with a soundtrack of p

#### Save the result logs

In [10]:
with open('output/last_trial_reflect_cot_no_context/100_questions_5_trials.txt', 'w') as f:
    f.write(log)
save_agents(agents, 'output/last_trial_cot_no_context/100_questions_6_trials')